# Classification and regression models

In [1]:
import rebound
from spock import FeatureClassifier
feature_model = FeatureClassifier()

sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-5, P=1., e=0.03, pomega=2., l=0.5)
sim.add(m=1.e-5, P=1.2, e=0.03, pomega=3., l=3.)
sim.add(m=1.e-5, P=1.5, e=0.03, pomega=1.5, l=2.)
sim.move_to_com()

print(feature_model.predict_stable(sim))

0.06591137


In [2]:
from spock import DeepRegressor
deep_model = DeepRegressor()

median, lower, upper = deep_model.predict_instability_time(sim, samples=10000)
print(int(median))

229709


In [3]:
from spock import CollisionMergerClassifier
class_model = CollisionMergerClassifier()

prob_12, prob_23, prob_13 = class_model.predict_collision_probs(sim)

print('Probability planet 1 collides with 2:', prob_12)
print('Probability planet 2 collides with 3:', prob_23)
print('Probability planet 1 collides with 3:', prob_13)

Probability planet 1 collides with 2: 0.27385116
Probability planet 2 collides with 3: 0.49274087
Probability planet 1 collides with 3: 0.23340799


In [4]:
from spock import CollisionOrbitalOutcomeRegressor
reg_model = CollisionOrbitalOutcomeRegressor()

new_sim = reg_model.predict_collision_outcome(sim, collision_inds=[2, 3])

print('Predicted P1:', new_sim.particles[1].P)
print('Predicted e1:', new_sim.particles[1].e, '\n')
print('Predicted P2:', new_sim.particles[2].P)
print('Predicted e2:', new_sim.particles[2].e)

Predicted P1: 1.2250393057947184
Predicted e1: 0.05889093054898333 

Predicted P2: 1.4126459270205634
Predicted e2: 0.029298967242683358


# Giant impact emulator

In [1]:
import rebound
from spock import GiantImpactPhaseEmulator, CollisionOrbitalOutcomeRegressor, CollisionMergerClassifier

In [2]:
sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-5, P=1, e=0.03, pomega=2., l=0.5)
sim.add(m=1.e-5, P=1.2, e=0.03, pomega=3., l=3.)
sim.add(m=1.e-5, P=1.5, e=0.03, pomega=1.5, l=2.)
sim.add(m=1.e-5, P=2.2, e=0.03, pomega=0.5, l=5.0)
sim.add(m=1.e-5, P=2.5, e=0.03, pomega=5.0, l=1.5)
sim.move_to_com()

In [18]:
emulator = GiantImpactPhaseEmulator(seed=6)

In [19]:
%%time
sims = emulator.predict(sim)

CPU times: user 1.36 s, sys: 683 ms, total: 2.04 s
Wall time: 1.13 s


In [20]:
ps = sims[0].particles
print('t =', sims[0].t, '\n')
for i in range(1, len(ps)):
    print('Predicted m' + str(i), '=', ps[i].m)
    print('Predicted P' + str(i), '=', ps[i].P)
    print('Predicted e' + str(i), '=', ps[i].e, '\n')

t = 999999999.9999993 

Predicted m1 = 2e-05
Predicted P1 = 1.1406315833630705
Predicted e1 = 0.027527879033166853 

Predicted m2 = 1e-05
Predicted P2 = 2.243560590440474
Predicted e2 = 0.03273085504984949 

Predicted m3 = 2e-05
Predicted P3 = 3.517799251077447
Predicted e3 = 0.01492289047762144 

